<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-1-public/blob/master/C2/W4/ungraded_lab/C2_W4_Lab_1_multi_class_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://raw.githubusercontent.com/doantronghieu/DEEP-LEARNING/main/helper_DL.py
!pip install colorama
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size':15})
import seaborn           as sns
sns.set()
import helper_DL as helper

# Ungraded Lab: Multi-class Classifier

In this lab, you will look at how to build a model to distinguish between more than two classes. The code will be similar to the ones you've been using before with a few key changes in the model and in the training parameters. Let's dive in!


## Download and Prepare the Dataset

You will be using the [Rock-Paper-Scissors dataset](http://www.laurencemoroney.com/rock-paper-scissors-dataset/), a gallery of hands images in Rock, Paper, and Scissors poses.

In [ ]:
# Download the train set
!wget https://storage.googleapis.com/tensorflow-1-public/course2/week4/rps.zip
# Download the test set
!wget https://storage.googleapis.com/tensorflow-1-public/course2/week4/rps-test-set.zip

In [ ]:
import zipfile

# Extract the archive
local_zip = './rps.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('tmp/rps-train')
zip_ref.close()

local_zip = './rps-test-set.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('tmp/rps-test')
zip_ref.close()

As usual, you will assign the directory names into variables and look at the filenames as a sanity check.

In [ ]:
import os

base_dir = 'tmp/rps-train/rps'

rock_dir     = os.path.join(base_dir, 'rock')
paper_dir    = os.path.join(base_dir, 'paper')
scissors_dir = os.path.join(base_dir, 'scissors')

rock_files     = os.listdir(rock_dir)
paper_files    = os.listdir(paper_dir)
scissors_files = os.listdir(scissors_dir)

print(f'Total training rock images:     {len(os.listdir(rock_dir))}')
print(f'Total training paper images:    {len(os.listdir(paper_dir))}')
print(f'Total training scissors images: {len(os.listdir(scissors_dir))}\n')

print(rock_files[:10])
print(paper_files[:10])
print(scissors_files[:10])

You can also inspect some of the images to see the variety in your model inputs.

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

pic_index = 2

next_rock     = [os.path.join(rock_dir, fname)     for fname in rock_files    [pic_index - 2:pic_index]]
next_paper    = [os.path.join(paper_dir, fname)    for fname in paper_files   [pic_index - 2:pic_index]]
next_scissors = [os.path.join(scissors_dir, fname) for fname in scissors_files[pic_index - 2:pic_index]]

for (i, img_path) in enumerate(next_rock + next_paper + next_scissors):
    img = mpimg.imread(img_path)
    plt.imshow(img)
    plt.axis('Off')
    plt.show()

## Build the model

You will then build your CNN. You will use 4 convolution layers with 64-64-128-128 filters then append a `Dropout` layer to avoid overfitting and some Dense layers for the classification. The output layer would be a 3-neuron dense layer activated by [Softmax](https://www.tensorflow.org/api_docs/python/tf/nn/softmax). You've seen this in Course 1 when you were training with Fashion MNIST. It scales your output to a set of probabilities that add up to 1. The order of this 3-neuron output would be `paper`-`rock`-`scissors` (e.g. a `[0.8 0.2 0.0]` output means the model is prediciting 80% probability for paper and 20% probability for rock.

You can examine the architecture with `model.summary()` below.

In [ ]:
import tensorflow as tf
import tensorflow.keras as tfk
from tensorflow import nn
from tensorflow.keras import layers, optimizers, losses, models, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
model = models.Sequential([
    # The input shape is the desired size of the image 150x150 with 3 bytes color
    # The first convolution
    layers.Conv2D(64, (3, 3), activation = nn.relu, input_shape = (150, 150, 3)),
    layers.MaxPooling2D(2, 2),

    # The second convolution
    layers.Conv2D(64, (3, 3), activation = nn.relu),
    layers.MaxPooling2D(2, 2),

    # The third convolution
    layers.Conv2D(128, (3, 3), activation = nn.relu),
    layers.MaxPooling2D(2, 2),

    # The fourth convolution
    layers.Conv2D(128, (3, 3), activation = nn.relu),
    layers.MaxPooling2D(2, 2),

    # Flatten the results to feed into a DNN
    layers.Flatten(),
    layers.Dropout(0.5),

    # 512 neuron hidden layer
    layers.Dense(512, activation = nn.relu),
    layers.Dense(3, activation = nn.softmax)
])

model.summary()

You will then compile the model. The key change here is the `loss` function. Whereas before you were using `binary_crossentropy` for 2 classes, you will change it to [categorical_crossentropy](https://keras.io/api/losses/probabilistic_losses/#categoricalcrossentropy-function) to extend it to more classes.

In [ ]:
# Set the training parameters
model.compile(loss = losses.categorical_crossentropy,
              optimizer = optimizers.RMSprop(),
              metrics = ['accuracy'])

## Prepare the ImageDataGenerator

You will prepare the generators as before. You will set the training set up for data augmentation so it can mimick other poses that the model needs to learn.

In [ ]:
TRAINING_DIR   = 'tmp/rps-train/rps'
VALIDATION_DIR = 'tmp/rps-test/rps-test-set'

training_datagen = ImageDataGenerator(rescale = 1. / 255,
                                      rotation_range = 40,
                                      width_shift_range = 0.2,
                                      height_shift_range = 0.2,
                                      shear_range = 0.2,
                                      zoom_range = 0.2,
                                      horizontal_flip = True,
                                      fill_mode = 'nearest')

validation_datagen = ImageDataGenerator(rescale = 1. / 255)

train_generator = training_datagen.flow_from_directory(TRAINING_DIR,
                                                       target_size = (150, 150),
                                                       class_mode = 'categorical',
                                                       batch_size = 126)

validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,
                                                              target_size = (150, 150),
                                                              class_mode = 'categorical',
                                                              batch_size = 126)

## Train the model and evaluate the results

You will train for 25 epochs and evaludate the results afterwards. Observe how both the training and validation accuracy are trending upwards. This is a good indication that the model is not overfitting to only your training set.



In [ ]:
# Train the model
history = model.fit(train_generator, epochs = 25, steps_per_epoch = 20,
                    validation_data = validation_generator, validation_steps = 3,
                    verbose = 1)

In [ ]:
helper.plot_history_curves(history)

# Model Prediction

You should be able to upload an image here and have it classified without crashing. This codeblock will only work in Google Colab, however. You can use your own images or use the ones available [here](https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps-validation.zip)

**Important Note:** Due to some compatibility issues, the following code block will result in an error after you select the images(s) to upload if you are running this notebook as a `Colab` on the `Safari` browser. For all other browsers, continue with the next code block and ignore the next one after it.

_For Safari users: please comment out or skip the code block below, uncomment the next code block and run it._

In [ ]:
def predict_softmax(target_size, model, class_names):
  """
  Args: 
    target_size (list)  (int)
    class_names (tuple) (str)
  """
  from google.colab import files
  from keras.preprocessing import image
  
  uploaded = files.upload()

  for fn in uploaded.keys():

      # Predicting images
      path = '/content/' + fn

      img = image.load_img(path, target_size=(target_size[0], target_size[1]))
      img = image.img_to_array(img)
      img = img / 255
      img = np.expand_dims(img, axis=0)

      images = np.vstack([img])
      classes = model.predict(images, batch_size=10)
      
      print(fn)
      print(classes)

In [ ]:
predict_softmax(target_size=(150, 150), model=model, class_names=['rock', 'paper', 'scissors'])

## Wrap Up

That concludes this short exercise on the multi-class classifiers. You saw that with just a few changes, you were able to convert your binary classifiers to predict more classes. You used the same techniques for data and model preparation and were able to get relatively good results in just 25 epochs. For practice, you can search for other datasets (e.g. [here](https://archive.ics.uci.edu/ml/datasets.php) with more classes and revise the model to accomodate it. Try to experiment with different layers and data augmentation techniques to improve your metrics.